In [12]:

import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize
import cv2
import os
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

tf.random.set_seed(42)
np.random.seed(42)

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [13]:
# SAFE CLEAR SESSION
import tensorflow as tf
tf.keras.backend.clear_session()
import gc
gc.collect()



0

In [14]:

class Config:
    IMG_SIZE = (224, 224)
    BATCH_SIZE = 8
    EPOCHS = 4
    NUM_CLASSES = 4
    LEARNING_RATE = 0.0001
    DATA_PATH = '/kaggle/input/imagesoasis/Data' 

config = Config()

def check_dataset_structure():
    if os.path.exists(config.DATA_PATH):
        classes = os.listdir(config.DATA_PATH)
        print("📁 Dataset structure:")
        for class_name in classes:
            class_path = os.path.join(config.DATA_PATH, class_name)
            if os.path.isdir(class_path):
                num_images = len([f for f in os.listdir(class_path) if f.endswith(('.jpg', '.png', '.jpeg'))])
                print(f"   {class_name}: {num_images} images")
    else:
        print("❌ Dataset path not found. Please check the path.")

check_dataset_structure()

📁 Dataset structure:
   Non Demented: 67222 images
   Very mild Dementia: 13725 images
   Moderate Dementia: 488 images
   Mild Dementia: 5002 images


In [15]:

def create_data_generators():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=15,  
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        zoom_range=0.2,
        brightness_range=[0.8, 1.2],
        validation_split=0.2 
    )
    
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = train_datagen.flow_from_directory(
        config.DATA_PATH,
        target_size=config.IMG_SIZE,
        batch_size=config.BATCH_SIZE,
        class_mode='categorical',
        subset='training',
        shuffle=True
    )
    
    val_generator = train_datagen.flow_from_directory(
        config.DATA_PATH,
        target_size=config.IMG_SIZE,
        batch_size=config.BATCH_SIZE,
        class_mode='categorical',
        subset='validation',
        shuffle=False
    )
    
    return train_generator, val_generator

print("🔄 Creating data generators...")
train_generator, val_generator = create_data_generators()

print(f"📊 Classes: {train_generator.class_indices}")
print(f"📈 Training samples: {train_generator.samples}")
print(f"📉 Validation samples: {val_generator.samples}")

🔄 Creating data generators...


KeyboardInterrupt: 

In [ ]:

import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

tf.keras.backend.clear_session()

def create_enhanced_densenet():
    base_model = DenseNet121(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    
    for layer in base_model.layers:
        layer.trainable = False
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu', 
              kernel_regularizer=tf.keras.regularizers.l2(0.001))(x) 
    x = Dropout(0.6)(x) 
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu', 
              kernel_regularizer=tf.keras.regularizers.l2(0.001))(x) 
    x = Dropout(0.5)(x)  
    x = Dense(128, activation='relu')(x)
    predictions = Dense(config.NUM_CLASSES, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    optimizer = Adam(learning_rate=config.LEARNING_RATE)
    
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy', 'precision', 'recall']
    )
    
    return model

print("🧠 Creating ENHANCED DenseNet model with stronger regularization...")
model = create_enhanced_densenet()
print("✅ Enhanced model created successfully!")

In [ ]:

def create_callbacks():
    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=2,
            restore_best_weights=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=1,
            min_lr=1e-7,
            verbose=1
        ),
        ModelCheckpoint(
            'best_densenet_model.h5',
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        )
    ]
    return callbacks

print("⏰ Callbacks configured:")
print("   - Early Stopping (patience: 7)")
print("   - Reduce LR on Plateau")
print("   - Model Checkpointing")

In [ ]:
print("🎯 Starting model training WITH CLASS BALANCING...")

from sklearn.utils.class_weight import compute_class_weight
import numpy as np

y_train = train_generator.classes
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = dict(enumerate(class_weights))

print("📊 Applying class weights to address imbalance:")
class_names = list(train_generator.class_indices.keys())
for i, (class_name, weight) in enumerate(zip(class_names, class_weights)):
    samples = np.sum(y_train == i)
    print(f"   {class_name}: {samples} samples → weight: {weight:.2f}x")

callbacks = create_callbacks()

history = model.fit(
    train_generator,
    epochs=config.EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks,
    class_weight=class_weight_dict,  
    verbose=1
)

print("✅ Training completed with class balancing!")

In [ ]:
# Training history
def plot_training_history(history):
    plt.figure(figsize=(15, 5))
    
    # Accuracy plot 
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
    plt.title('Model Accuracy', fontsize=14, fontweight='bold')
    plt.ylabel('Accuracy', fontsize=12)
    plt.xlabel('Epoch', fontsize=12)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss', linewidth=2)
    plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    plt.title('Model Loss', fontsize=14, fontweight='bold')
    plt.ylabel('Loss', fontsize=12)
    plt.xlabel('Epoch', fontsize=12)
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

print("📈 Plotting training history...")
plot_training_history(history)

In [ ]:
def comprehensive_evaluation(model, val_generator):
    
    val_generator.reset()
    y_true = val_generator.classes
    y_pred_proba = model.predict(val_generator, verbose=1)
    y_pred = np.argmax(y_pred_proba, axis=1)
    
    
    class_labels = list(val_generator.class_indices.keys())
    
    # 1. Classification Report
    print("=" * 60)
    print("COMPREHENSIVE CLASSIFICATION REPORT")
    print("=" * 60)
    print(classification_report(y_true, y_pred, target_names=class_labels))
    
    # 2. Confusion Matrix
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_labels, yticklabels=class_labels)
    plt.title('Confusion Matrix', fontweight='bold')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    # 3. Normalized Confusion Matrix
    plt.subplot(1, 3, 2)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues',
                xticklabels=class_labels, yticklabels=class_labels)
    plt.title('Normalized Confusion Matrix', fontweight='bold')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    # 4. ROC Curve (One-vs-Rest for multi-class)
    plt.subplot(1, 3, 3)
    y_true_bin = label_binarize(y_true, classes=[0, 1, 2, 3])
    
    fpr = {}
    tpr = {}
    roc_auc = {}
    
    for i in range(config.NUM_CLASSES):
        fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_pred_proba[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
        plt.plot(fpr[i], tpr[i], label=f'{class_labels[i]} (AUC = {roc_auc[i]:.2f})')
    
    plt.plot([0, 1], [0, 1], 'k--', linewidth=1)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Multi-class ROC Curve', fontweight='bold')
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 5. Additional Metrics
    accuracy = np.mean(y_true == y_pred)
    print(f"\n📊 Overall Accuracy: {accuracy:.4f}")
    
    # Per-class accuracy
    class_accuracy = {}
    print("\n📈 Per-class Accuracy:")
    for i, label in enumerate(class_labels):
        class_mask = y_true == i
        if np.sum(class_mask) > 0:  # Avoid division by zero
            class_accuracy[label] = np.mean(y_pred[class_mask] == i)
            print(f"   {label}: {class_accuracy[label]:.4f}")
    
    return y_true, y_pred, y_pred_proba

print("📊 Performing comprehensive evaluation...")
y_true, y_pred, y_pred_proba = comprehensive_evaluation(model, val_generator)

In [ ]:

def visualize_predictions(model, val_generator, num_samples=8):
    val_generator.reset()
    x_batch, y_batch = next(val_generator)
    y_pred = model.predict(x_batch)
    
    class_labels = list(val_generator.class_indices.keys())
    
    plt.figure(figsize=(15, 10))
    for i in range(num_samples):
        plt.subplot(2, 4, i+1)
        plt.imshow(x_batch[i])
        true_label = class_labels[np.argmax(y_batch[i])]
        pred_label = class_labels[np.argmax(y_pred[i])]
        confidence = np.max(y_pred[i])
        
        color = 'green' if true_label == pred_label else 'red'
        plt.title(f'True: {true_label}\nPred: {pred_label}\nConf: {confidence:.2f}', 
                 color=color, fontsize=10, fontweight='bold')
        plt.axis('off')
    
    plt.tight_layout()
    plt.suptitle('Sample Predictions (Green=Correct, Red=Incorrect)', fontsize=14, fontweight='bold')
    plt.show()

print("👀 Visualizing sample predictions...")
visualize_predictions(model, val_generator)

In [ ]:

print("💾 Saving model...")
model.save('alzheimer_densenet_final.h5')

final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]

print("\n" + "="*50)
print("🎯 FINAL TRAINING RESULTS")
print("="*50)
print(f"📈 Final Training Accuracy: {final_train_acc:.4f}")
print(f"📉 Final Validation Accuracy: {final_val_acc:.4f}")
print(f"📈 Final Training Loss: {final_train_loss:.4f}")
print(f"📉 Final Validation Loss: {final_val_loss:.4f}")

if len(history.history['accuracy']) > 1:
    initial_acc = history.history['accuracy'][0]
    improvement = final_train_acc - initial_acc
    print(f"📊 Accuracy Improvement: +{improvement:.4f}")

print("\n✅ Pipeline completed successfully!")
print("🚀 Model saved as 'alzheimer_densenet_final.h5'")

In [ ]:

def additional_analysis():
    print("🔍 Additional Analysis for Research Paper:")
    print("-" * 50)
    
    trainable_params = np.sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
    non_trainable_params = np.sum([tf.keras.backend.count_params(w) for w in model.non_trainable_weights])
    total_params = trainable_params + non_trainable_params
    
    print(f"📐 Model Parameters:")
    print(f"   Trainable: {trainable_params:,}")
    print(f"   Non-trainable: {non_trainable_params:,}")
    print(f"   Total: {total_params:,}")
    
    print(f"\n📊 Class Distribution:")
    for class_name, class_idx in train_generator.class_indices.items():
        count = np.sum(y_true == class_idx)
        print(f"   {class_name}: {count} samples")

additional_analysis()

In [ ]:
print("🔬 Starting ablation experiments...")

def build_ablation_model(dropout1=0.6, dropout2=0.5):
    base_model = DenseNet121(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3)
    )
    for layer in base_model.layers:
        layer.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu',
              kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = Dropout(dropout1)(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu',
              kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = Dropout(dropout2)(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(config.NUM_CLASSES, activation='softmax')(x)
    model_ab = Model(inputs=base_model.input, outputs=predictions)
    optimizer = Adam(learning_rate=config.LEARNING_RATE)
    model_ab.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy',
                 tf.keras.metrics.Precision(name='precision'),
                 tf.keras.metrics.Recall(name='recall')]
    )
    return model_ab

def create_no_aug_generators():
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    train_gen = datagen.flow_from_directory(
        config.DATA_PATH,
        target_size=config.IMG_SIZE,
        batch_size=config.BATCH_SIZE,
        class_mode='categorical',
        subset='training'
    )
    val_gen = datagen.flow_from_directory(
        config.DATA_PATH,
        target_size=config.IMG_SIZE,
        batch_size=config.BATCH_SIZE,
        class_mode='categorical',
        subset='validation',
        shuffle=False
    )
    return train_gen, val_gen

def run_ablation_experiment(name,
                            use_class_weights=True,
                            use_augmentation=True,
                            dropout1=0.6,
                            dropout2=0.5,
                            epochs=2):
    tf.keras.backend.clear_session()
    if use_augmentation:
        train_gen = train_generator
        val_gen = val_generator
    else:
        train_gen, val_gen = create_no_aug_generators()
    model_ab = build_ablation_model(dropout1=dropout1, dropout2=dropout2)
    callbacks = create_callbacks()
    cw = class_weight_dict if use_class_weights else None
    history_ab = model_ab.fit(
        train_gen,
        epochs=epochs,
        validation_data=val_gen,
        callbacks=callbacks,
        class_weight=cw,
        verbose=1
    )
    results = model_ab.evaluate(val_gen, verbose=0)
    metrics = dict(zip(model_ab.metrics_names, results))
    metrics["experiment"] = name
    metrics["use_class_weights"] = use_class_weights
    metrics["use_augmentation"] = use_augmentation
    metrics["dropout1"] = dropout1
    metrics["dropout2"] = dropout2
    metrics["epochs"] = epochs
    return metrics, history_ab

baseline_metrics, baseline_history = run_ablation_experiment(
    name="Baseline_full",
    use_class_weights=True,
    use_augmentation=True,
    dropout1=0.6,
    dropout2=0.5,
    epochs=4
)

no_cw_metrics, no_cw_history = run_ablation_experiment(
    name="No_class_weights",
    use_class_weights=False,
    use_augmentation=True,
    dropout1=0.6,
    dropout2=0.5,
    epochs=2
)

no_aug_metrics, no_aug_history = run_ablation_experiment(
    name="No_augmentation",
    use_class_weights=True,
    use_augmentation=False,
    dropout1=0.6,
    dropout2=0.5,
    epochs=2
)

low_do_metrics, low_do_history = run_ablation_experiment(
    name="Dropout_0_3",
    use_class_weights=True,
    use_augmentation=True,
    dropout1=0.3,
    dropout2=0.3,
    epochs=2
)

all_results = [
    baseline_metrics,
    no_cw_metrics,
    no_aug_metrics,
    low_do_metrics
]

ablation_results_df = pd.DataFrame(all_results)
print(ablation_results_df)
